Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.decomposition import TruncatedSVD
import re
import spacy
from spacy.tokenizer import Tokenizer

In [2]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
# making observations on data quality, \n newlines,  \x?, 
train.description.iloc[2]

'\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding. '

In [5]:
train['description'] = train['description'].replace(r".\\x[a-z].?", ' ', regex=True )

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
STOP_WORDS = nlp.Defaults.stop_words.union(["\n"])

def get_lemmas(text):
  """Returns `lemmas` in a list"""
  tokens = []
  str1= " "
  doc = nlp(text)

  for token in doc:
    if (token.is_stop == False) & (token.is_punct == False) & (token.text.lower() not in STOP_WORDS):
      tokens.append(token.lemma_)

  return (str1.join(tokens))

In [8]:
def wrangle(X):
    # wrangles the thign
    
    X = X.apply(get_lemmas)
    
    
    return X

In [9]:
target = 'ratingCategory'
Y_train = np.array(train[target])
X_train = train['description']

In [10]:
X_train = wrangle(X_train)

In [11]:
X_train

0       whisky batch leftover barrel return warehouse ...
1       uncommon exclusive bottling 6 year old cask st...
2       release port version Amrut Intermediate Sherry...
3       41 year old single cask age sherry butt intera...
4       herbal nose aroma dry tarragon parsley dill ch...
                              ...                        
4082    lie beneath surface Dewar blend finish virgin ...
4083    6 7 year maturation bourbon cask spend month c...
4084    bright delicate approachable showstopper deliv...
4085    call pitmaster dram nose muscular peat smoke p...
4086    spicy sultana greengage plum toffee new leathe...
Name: description, Length: 4087, dtype: object

### Define Pipeline Components

In [12]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,1))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [13]:
X_train.shape

(4087,)

In [15]:
Y_train.shape

(4087,)

In [16]:
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [17]:
lsi = Pipeline([("vect", vect),
                ("svd", svd)])
pipe = Pipeline([("lsi", lsi),
                ("clf", clf)])

In [21]:
alt_pipe = Pipeline([("vect", vect),
                    ("clf", clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [23]:
parameters = {
  'lsi__svd__n_components': [100],
  'vect__max_df': (1.0),
  'vect__min_df': (10),
  'vect__max_features': (1000),
  'clf__n_estimators':(90),
  'clf__max_depth':(40)
}

grid_search = GridSearchCV(alt_pipe, parameters, n_jobs=-1, cv=5, verbose=1)
grid_search.fit(X_train, Y_train)

ValueError: Parameter values for parameter (vect__min_df) need to be a sequence(but not a string) or np.ndarray.

In [19]:
grid_search.best_params_

{'clf__max_depth': 40,
 'clf__n_estimators': 90,
 'lsi__svd__n_components': 100,
 'lsi__vect__max_df': 1.0,
 'lsi__vect__max_features': 1000,
 'lsi__vect__min_df': 10}

In [20]:
grid_search.best_score_

0.7276753463233908

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [166]:
X_test = test['description']

In [167]:
X_test = wrangle(X_test)

In [28]:
# Predictions on test sample
pred = grid_search.predict(X_test)

In [29]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [30]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [31]:
subNumber = 0

In [33]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [1]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

In [ ]:
Sentiment analysis is the voice in which text can be read. its the inflections used in text based on word choice.

Document classification reffers to how documents are organised based on their attributes. Sentiment analysis is only one attribute of the documents being classified.


Common applications of sentiment analysis are in online reviews and rating systems.